In [243]:
import cv2
import sys
import math
import cv2 as cv
import numpy as np

In [323]:
### 0. 이미지 불러오기
#img = "/Users/yu/tf39_test/실습/자율주행_신호등인식/train/데이터/차선_샘플/a.jpg"
img = "/Users/yu/tf39_test/실습/자율주행_신호등인식/train/데이터/test/15790582.jpg"
A = cv2.imread(img, cv2.IMREAD_COLOR)

cv2.imshow("A",A)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [324]:
# ROI 지정(중앙쪽으로) 1520*580
A_roi = A.copy()

X=200; Y=500; W=1520; H=580;
A_roi = A_roi[Y:Y+H, X:X+W]

cv2.imshow("A_roi",A_roi)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [325]:
A_roi.shape[:3]

(580, 1520, 3)

In [326]:
### 1. 차선 진행 방향 고려해서 roi 지정(사다리꼴)
# roi 사다리꼴 마스크 설정
mask1 = np.zeros(A_roi.shape[:3],dtype = np.uint8)

polygon = np.array([[500,0],[1020,0],[1450,580],[70,580]])
white = (255,255,255)

mask1 = cv2.fillPoly(mask1,[polygon],white)

cv2.imshow("mask1",mask1)
cv2.imshow("original",A_roi)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [327]:
# roi 사다리꼴 자르기
a_copy = A_roi.copy()
a_result = cv2.bitwise_and(a_copy, mask1)

cv2.imshow('a_result',a_result) # 본래의 색인 마스크

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [141]:
### 2. hsv에서 흰색 차선 후보 검출, hsv에서 노랑색 차선 후보 검출

In [316]:
## 노란색 부분 추출
a = a_result.copy()
# hsv 영역 전환
a2 = cv2.cvtColor(a,cv2.COLOR_BGR2HSV)

# hsv 색 범위 지정2
low = (20, 20, 100)
up = (31, 255, 255)

a2_mask = cv2.inRange(a2, low, up)
a2_result = cv2.bitwise_and(a,a, mask = a2_mask)

cv2.imshow('color',a) # 원본
cv2.imshow('mask',a2_mask) # 마스크
cv2.imshow('result',a2_result) # 본래의 색인 마스크

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [331]:
## 흰색 부분 추출
a = a_result.copy()
# hsv 영역 전환
a2 = cv2.cvtColor(a,cv2.COLOR_BGR2HSV)

# bgr 색 범위 지정
#low = (0, 255, 255)
#up = (200, 255, 255)

# hsv 색 범위 지정2
low = (0, 0, 150)
up = (250, 90, 250)

a2_mask = cv2.inRange(a2, low, up)
a2_result = cv2.bitwise_and(a,a, mask = a2_mask)

cv2.imshow('color',a) # 원본
cv2.imshow('mask',a2_mask) # 마스크
cv2.imshow('result',a2_result) # 본래의 색인 마스크

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [332]:
### 3. 캐니 활용해서 윤곽선 드러내기
# 캐니 추출

a2_canny = cv2.Canny(a2_result, 50 , 150)

cv2.imshow("a2_canny",a2_canny)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [ ]:
'''# 일반적 허프 선 변환
src = a2_result
dst = a2_canny
cdst = cv.cvtColor(a2_canny, cv.COLOR_GRAY2BGR)

cdstP = np.copy(cdst)

lines = cv2.HoughLines(dst, 1, np.pi/180, 50, srn = 100, stn = 200, min_theta = 0, max_theta = np.pi)'''

In [255]:
'''# 확률적 허프 선 변환
src = a2_result
dst = a2_canny
cdst = cv.cvtColor(a2_canny, cv.COLOR_GRAY2BGR)

cdstP = np.copy(cdst)

linesP = cv.HoughLinesP(dst, 1, np.pi/180, 50, None, 50, 10)

if linesP is not None:
    for i in range(0, len(linesP)):
        l = linesP[i][0]
        cv.line(cdstP, (l[0], l[1]), (l[2], l[3]), (0, 0, 255), 2, cv.LINE_AA)
        
        
cv.imshow("확률적 허프 선 변환", cdstP)

cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)'''

-1

In [ ]:
# img :
# rho :
# theta :
# threshold : (점 임계값)
# lines : 검출된 넘파이 배열 선 좌표들 (생략) None
# minlinelength : (선 최소 길이)
# maxLineGap : (선사이 최대 허용간격)

In [335]:
### 4. 하프변환P로 차선 긋기
# 검출된 데이터 없으면 오류 뜨므로 수정 요함
r = []
l = []
src = a2_result
dst = a2_canny
cdst = cv.cvtColor(a2_canny, cv.COLOR_GRAY2BGR)

cdstP = np.copy(cdst)

linesP = cv2.HoughLinesP(dst, 1, np.pi/180, 50, None, 50, 10)
#if linesP != None: 리스트에 데이터 없으면 오류나서 추가해야함 근데 데이터 있으면 오류남
for a in linesP:
    for b in a: # [] 가 두개 겹쳐있음
        if (b[2]-b[0]) != 0: # 분모가 0이면 나눌 수 없어
            radian = math.atan((b[3]-b[1])/(b[2]-b[0]))
            degree = radian * 180 / math.pi
            if degree >= 30 and degree <= 80: # 오른쪽
                r.append(b)
                cv2.line(cdstP, (b[0], b[1]), (b[2], b[3]), (0, 0, 255), 2, cv2.LINE_AA)
            elif degree >= -80 and degree <= -30: # 왼쪽
                l.append(b)
                cv2.line(cdstP, (b[0], b[1]), (b[2], b[3]), (0, 0, 255), 2, cv2.LINE_AA)

cv.imshow("houghP", cdstP)

cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [ ]:
### 여기까지 하면 차선 양옆들 검출 성공

In [320]:
# 확인
r

[]

In [321]:
# 확인2
l

[]

In [336]:
### 5. roi 이미지에서 선그리기 (+각 선들 특징에 맞게 위치 저장)
a_copy = A_roi.copy()

# r 에서 x0 최소값이 있는 인덱스 i 찾기
least = []
for x in r:
    least.append(x[0])
    
least_a = min(least)
i = least.index(least_a)

# r 에서 x0 최대값이 있는 인덱스 i2 찾기
most_a = max(least)
i2 = least.index(most_a)

# l 에서 x1 최소값이 있는 인덱스 i3 찾기
least2 = []
for x in l:
    least2.append(x[2])
    
least_b = min(least2)
i3 = least2.index(least_b)

# l 에서 x1 최대값이 있는 인덱스 i4 찾기  
most_b = max(least2)
i4 = least2.index(most_b)


cv2.line(a_copy, (r[i][0], r[i][1]), (r[i][2], r[i][3]), (0, 0, 255), 2, cv2.LINE_AA)
cv2.line(a_copy, (r[i2][0], r[i2][1]), (r[i2][2], r[i2][3]), (0, 0, 255), 2, cv2.LINE_AA)

cv2.line(a_copy, (l[i3][0], l[i3][1]), (l[i3][2], l[i3][3]), (0, 0, 255), 2, cv2.LINE_AA)
cv2.line(a_copy, (l[i4][0], l[i4][1]), (l[i4][2], l[i4][3]), (0, 0, 255), 2, cv2.LINE_AA)


cv.imshow("a_copy", a_copy)

cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [338]:
### 6. l, r 선길게 4개 긋기
a_copy2 = A_roi.copy()

# 기울기 m
# y절편 n

## r
m = (r[i][3]-r[i][1])/(r[i][2]-r[i][0])
n = r[i][1] - (m * r[i][0])

## r2
m2 = (r[i2][3]-r[i2][1])/(r[i2][2]-r[i2][0])
n2 = r[i2][1] - (m2 * r[i2][0])

## l1
m3 = (l[i3][3]-l[i3][1])/(l[i3][2]-l[i3][0])
n3 = l[i3][1] - (m3 * l[i3][0])

## l2
m4 = (l[i4][3]-l[i4][1])/(l[i4][2]-l[i4][0])
n4 = l[i4][1] - (m4 * l[i4][0])


cv2.line(a_copy2, (int(-n/m), 0),(int((580-n)/m), 580), (0, 0, 255), 2, cv2.LINE_AA)
cv2.line(a_copy2, (int(-n2/m2), 0),(int((580-n2)/m2), 580), (0, 0, 255), 2, cv2.LINE_AA)
cv2.line(a_copy2, (int(-n3/m3), 0),(int((580-n3)/m3), 580), (0, 0, 255), 2, cv2.LINE_AA)
cv2.line(a_copy2, (int(-n4/m4), 0),(int((580-n4)/m4), 580), (0, 0, 255), 2, cv2.LINE_AA)

cv2.imshow("a_copy2", a_copy2)

cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [ ]:
### 하얀색, 노란색 차선 두개 합쳐서 출력필요



In [ ]:
####### 테스트한 코드 영역 ##########

In [295]:
'''print((r[i][0], r[i][1]), (r[i][2], r[i][3]))
print(m)
print(n)
print("==========")
print(-n/m)
print((580-n)/m)
print("==========")
print((int(-n/m), 0),(int((580-n)/m), 580))'''

(1147, 420) (1220, 498)
1.0684931506849316
-805.5616438356165
753.9230769230769
1296.7435897435896
(753, 0) (1296, 580)


In [308]:
'''print((r[i3][0], r[i3][1]), (r[i3][2], r[i3][3]))
print(m3)
print(n3)
print("==========")
print(-n3/m3)
print((580-n3)/m3)
print("==========")
print((int(-n3/m3), 0),(int((580-n3)/m3), 580))'''

(1241, 467) (1303, 529)


'print(m3)\nprint(n3)\nprint("==========")\nprint(-n3/m3)\nprint((580-n3)/m3)\nprint("==========")\nprint((int(-n3/m3), 0),(int((580-n3)/m3), 580))'

In [126]:
# 다중 리스트에서 내림차순으로 정렬(다른 방법)

'''l2 = l
l2.sort(key=lambda x:x[2])
print(l2)'''

[array([261, 373, 342, 314], dtype=int32), array([249, 383, 349, 313], dtype=int32), array([284, 383, 367, 318], dtype=int32), array([290, 381, 375, 315], dtype=int32)]


In [100]:
'''radian = math.atan(3)

degree = radian * 180/ math.pi

print(degree)'''

71.56505117707799


In [135]:
'''for a in linesP:
    for b in a: # [] 가 두개 겹쳐있음
        radian = math.atan((b[3]-b[1])/(b[2]-b[0]))
        degree = radian * 180 / math.pi
        print(degree)'''

32.929854510612074
33.857110141660684
35.06834668029092
10.304846468766033
22.976944503188754
23.039435979903534
23.334041276709776
14.995079129175988
22.12194197188185
7.6668042618141765
9.91494567847495
34.91940201245768
23.317708386755065
23.19859051364819
45.0
10.304846468766033
22.932100437589785
12.030596096537868
8.820379552021057
10.738897100905442
22.027278669171338
11.093723011557849
12.957580094093132
32.92471141680646
45.0
18.758650347842234
30.96375653207352
-25.38790629109593
8.488943880504715
30.68517308959513
25.615484320642167
23.886499921634748
9.090276920822323
19.025606037568686
10.88552705465874
17.981429696964042
21.39981013208223
33.231711067979354
24.19320898728877
20.224859431168078
11.768288932020647
15.312818360806306
12.724355685422369
33.97046877601952
-73.15068858817492
10.823011226207099
32.39984017391932
29.623748751173814
8.498558879493682
-23.92250588301327
-87.68627750217578
15.524110996754256
34.00749241973227
25.13295499291334
12.094757077012103
45.